# parte 1
pegar os links de cada um dos conjuntos

In [1]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(60)
url = 'https://www.quintoandar.com.br/alugar/imovel/sao-paulo-sp-brasil'

In [2]:
driver.get(url)
driver.implicitly_wait(10)

wait = WebDriverWait(driver, timeout=5)

In [3]:
horas = []

for i in range(10):
    try:
        horas = driver.find_elements(By.XPATH, '//main/section[2]/div/div')
        wait.until(EC.element_to_be_clickable((By.XPATH, '//main/section[2]/div/div[last()-2]/button')))
        horas[-3].click()
        wait.until(EC.staleness_of(horas[-3]))

    except Exception as err:
        print(f'\nse fudeu em {i}: {err = }\n')

        if isinstance(err, StaleElementReferenceException):
            print("Attempting to recover from StaleElementReferenceException")
            wait.until(EC.element_to_be_clickable((By.XPATH, '//main/section[2]/div/div[last()-2]/button')))
        else:
            raise err

    print(f'list size on iteration {i}: {len(horas)}')

list size on iteration 0: 16
list size on iteration 1: 28
list size on iteration 2: 40
list size on iteration 3: 52
list size on iteration 4: 64
list size on iteration 5: 76
list size on iteration 6: 88
list size on iteration 7: 100
list size on iteration 8: 112
list size on iteration 9: 124


In [4]:
links = []
# os divs de interesse vao desde o segundo ate o anteantepenultimo
for i, div in enumerate(horas[1:-4]):
    current_link = div.find_element(by=By.TAG_NAME, value='a').get_property('href')
    print(f'link at iteration {i} {current_link}')
    links.append(current_link)

link at iteration 0 https://www.quintoandar.com.br/imovel/894484379/alugar/kitnet-1-quarto-moema-sao-paulo
link at iteration 1 https://www.quintoandar.com.br/imovel/894484864/alugar/apartamento-1-quarto-santo-amaro-sao-paulo
link at iteration 2 https://www.quintoandar.com.br/imovel/894484608/alugar/kitnet-1-quarto-vila-mariana-sao-paulo
link at iteration 3 https://www.quintoandar.com.br/imovel/894484260/alugar/apartamento-1-quarto-bela-vista-sao-paulo
link at iteration 4 https://www.quintoandar.com.br/imovel/892850671/alugar/apartamento-4-quartos-bosque-da-saude-sao-paulo
link at iteration 5 https://www.quintoandar.com.br/imovel/894480566/alugar/apartamento-1-quarto-bras-sao-paulo
link at iteration 6 https://www.quintoandar.com.br/imovel/893226330/alugar/apartamento-2-quartos-jurubatuba-sao-paulo
link at iteration 7 https://www.quintoandar.com.br/imovel/894484361/alugar/kitnet-1-quarto-bela-vista-sao-paulo
link at iteration 8 https://www.quintoandar.com.br/imovel/894481894/alugar/apart

# parte 2
pra cada conjunto, obter informaçoes de interess
e

In [9]:
data_info = {
    'suite_area'    : {
        'xpath' : '//main/section/div/div[1]/div/div[3]/section/div/div[3]/div/div/div[1]/div/div/p',
        'type'  : 'numeric'
    },
    'street'        : {
        'xpath' : '//main/section/div/div[1]/div/div[2]/div/div/div/div[1]/div/h4',
        'type'  : 'text'
    },
    'neighborhood'  : {
        'xpath' : '//main/section/div/div[1]/div/div[2]/div/div/div/div[1]/small',
        'type'  : 'text'
    },
    'condominium'   : {
        'xpath' : '//main/section/div/div[1]/div/div[3]/section/div/div[2]/div/ul/li[2]/div/div/p',
        'type'  : 'numeric'
    },
    'tax'           : {
        'xpath' : '//main/section/div/div[1]/div/div[3]/section/div/div[2]/div/ul/li[3]/div/div/p',
        'type'  : 'numeric'
    },
    'asking_price'  : {
        'xpath' : '//main/section/div/div[1]/div/div[3]/section/div/div[2]/div/ul/li[1]/div/div/p',
        'type'  : 'numeric'
    }
}

In [12]:
import pandas as pd


delta_y = 800
stock = []

for i, link in enumerate(links):
    driver.get(link)
    driver.implicitly_wait(10)
    ActionChains(driver) \
        .scroll_by_amount(0, delta_y) \
        .perform()

    raw_suite_data = {
        key: driver.find_element(by=By.XPATH, value=info['xpath']).text for key, info in data_info.items()
    }

    print(f'raw data on iteration {i} {raw_suite_data}')
    
    treated_suite_data = {
        key: int('0'+''.join(filter(str.isdecimal, value))) if data_info[key]['type'] == 'numeric' else value for key, value in raw_suite_data.items()
    }
    
    print(f'treated data on iteration {i} {treated_suite_data}')
    
    stock.append(treated_suite_data)

raw data on iteration 0 {'suite_area': '21 m²', 'street': 'Avenida dos Imarés', 'neighborhood': 'Moema, São Paulo', 'condominium': 'R$ 679', 'tax': 'R$ 29', 'asking_price': 'R$ 3.500'}
treated data on iteration 0 {'suite_area': 21, 'street': 'Avenida dos Imarés', 'neighborhood': 'Moema, São Paulo', 'condominium': 679, 'tax': 29, 'asking_price': 3500}
raw data on iteration 1 {'suite_area': '33 m²', 'street': 'Avenida Vereador José Diniz', 'neighborhood': 'Santo Amaro, São Paulo', 'condominium': 'R$ 540', 'tax': 'R$ 92', 'asking_price': 'R$ 3.600'}
treated data on iteration 1 {'suite_area': 33, 'street': 'Avenida Vereador José Diniz', 'neighborhood': 'Santo Amaro, São Paulo', 'condominium': 540, 'tax': 92, 'asking_price': 3600}
raw data on iteration 2 {'suite_area': '21 m²', 'street': 'Rua Humberto I', 'neighborhood': 'Vila Mariana, São Paulo', 'condominium': 'R$ 349', 'tax': 'R$ 70', 'asking_price': 'R$ 3.000'}
treated data on iteration 2 {'suite_area': 21, 'street': 'Rua Humberto I', '

In [13]:
pd.DataFrame(stock)

,suite_area,street,neighborhood,condominium,tax,asking_price
0,21,Avenida dos Imarés,"Moema, São Paulo",679,29,3500
1,33,Avenida Vereador José Diniz,"Santo Amaro, São Paulo",540,92,3600
2,21,Rua Humberto I,"Vila Mariana, São Paulo",349,70,3000
3,19,Rua Barata Ribeiro,"Bela Vista, São Paulo",488,75,2900
4,117,Rua Izar,"Bosque da Saúde, São Paulo",1755,654,3700
...,...,...,...,...,...,...
114,62,Rua Waldomiro Fonseca,"Vila Regina, São Paulo",560,0,1800
115,170,Rua Conselheiro Moreira de Barros,"Lauzane Paulista, São Paulo",0,248,4000
116,45,Rua Lucy Anna Carrozo La Torre,"Parada Xv de Novembro, São Paulo",0,22,800
117,120,Avenida Lourival Fontes,"Jardim Planalto, São Paulo",0,55,2500
